In [ ]:
import pandas as pd

# Maintenance CSV
maintenance_data = [
    [1,"2025-08-01","Engine A","Oil leakage","Replace oil seal and check pressure"],
    [2,"2025-08-02","Landing Gear B","Hydraulic failure","Inspect hydraulic lines and refill fluid"],
    [3,"2025-08-03","Fuselage C","Corrosion","Apply anti-corrosion treatment and inspect panels"],
    [4,"2025-08-04","Avionics D","Navigation error","Run diagnostic and update firmware"],
    [5,"2025-08-05","Engine E","Overheating","Replace thermostat and clean cooling system"]
]
df_maintenance = pd.DataFrame(maintenance_data, columns=["id","date","equipment","issue","procedure"])
df_maintenance.to_csv("maintenance.csv", index=False)

# Aircraft Taxonomy CSV
taxonomy_data = [
    [1,"Engine","Turbofan","High-bypass turbofan engine used in commercial jets"],
    [2,"Landing Gear","Main Gear","Retractable main landing gear assembly"],
    [3,"Fuselage","Body","Pressurized aircraft body structure"],
    [4,"Avionics","Navigation","Navigation system including GPS and INS"],
    [5,"Electrical","Power","Electrical power distribution system"],
    [6,"Hydraulics","Actuation","Hydraulic system controlling flaps and landing gear"],
    [7,"Fuel","Storage","Fuel tanks and fuel distribution system"],
    [8,"Cabin","Seats","Passenger seating and cabin layout"],
    [9,"Flight Controls","Elevators","Primary control surfaces for pitch"],
    [10,"Flight Controls","Ailerons","Primary control surfaces for roll"]
]
df_taxonomy = pd.DataFrame(taxonomy_data, columns=["id","category","subcategory","description"])
df_taxonomy.to_csv("aircrafttaxonomy.csv", index=False)

In [ ]:
## Ingestion pipeline to load data
import os
import json
import pandas as pd
import requests
import httpx
from sqlalchemy import create_engine, text
from langchain.docstore.document import Document
from langchain_postgres.vectorstores import PGVector
from cfenv import AppEnv
from dotenv import load_dotenv

load_dotenv()
for k, v in os.environ.items():
    if k in ["API_BASE", "API_KEY", "MODEL_NAME", "DATABASE_URL", "OPENAI_API_KEY"]:
        print(f"{k} = {v}")

In [ ]:
DATABASE_URL = os.getenv("DATABASE_URL")
engine = create_engine(DATABASE_URL)

# Test DB connection
with engine.connect() as conn:
    version = conn.execute(text("SELECT version();")).fetchone()
    print("Connected to:", version[0])

In [ ]:
from langchain.document_loaders import CSVLoader
loader_maintenance = CSVLoader(file_path="maintenance.csv")
docs_maintenance = loader_maintenance.load()

loader_taxonomy = CSVLoader(file_path="aircrafttaxonomy.csv")
docs_taxonomy = loader_taxonomy.load()

all_docs = docs_maintenance + docs_taxonomy

In [ ]:
from langchain_postgres import PGVector
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

# See docker command above to launch a postgres instance with pgvector enabled.
connection = os.getenv("DATABASE_URL")  # Uses psycopg3!
collection_name = "my_docs"

vector_store = PGVector(
    embeddings=embeddings,
    collection_name=collection_name,
    connection=connection,
    use_jsonb=True,
)
vector_store.add_documents(all_docs)
print(f"✅ Inserted {len(all_docs)} documents into the vectorstore!")

In [ ]:
# -----------------------------
# Inspect DB
# -----------------------------
query = text("SELECT * FROM langchain_pg_collection LIMIT 5;")
print(pd.read_sql(query, engine))

query2 = text("SELECT * FROM langchain_pg_embedding LIMIT 5;")
print(pd.read_sql(query2, engine))

In [ ]:
import os
import requests
import json
import httpx
from openai import OpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain.agents import initialize_agent, AgentType, load_tools
from langchain_core.tools import Tool
from langchain.tools import tool
from langchain_openai import OpenAIEmbeddings
from datetime import date
import warnings
import ssl
from langchain_community.embeddings import OllamaEmbeddings
from openai import OpenAI
from langchain.chains import RetrievalQA

# Initialize LLM with credentials from cfenv
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)
# Create a retriever from your vectorstore
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k":3})

# Build a RetrievalQA chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)

# Ask a question
query = "Which aircraft equipment has reported issues with hydraulic leaks?"
result = qa.run(query)
print(result)